In [1]:
from ultralytics import YOLO
import cv2
import os


In [2]:
model = YOLO("C:/Users/niwat maneewong/OneDrive/Desktop/First/homwork/CS/Thaifood_ObjectDetectionNClassification/datasets/food/runs/detect/train14/weights/best.pt")  # path to your trained weights


In [3]:
# crop 1 จานได้
def crop_from_results(results, image_paths, output_dir='cropped', conf_threshold=0.5, target_class=1):
    os.makedirs(output_dir, exist_ok=True)

    for idx, result in enumerate(results):
        image_path = image_paths[idx]
        image = cv2.imread(image_path)

        if image is None:
            print(f"❌ Failed to read {image_path}")
            continue

        # Extract boxes, class IDs, and confidences
        try:
            boxes = result.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2, conf]
            class_ids = result.boxes.cls.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
        except AttributeError:
            print(f"❌ result[{idx}] is missing .boxes or attributes")
            continue

        crop_count = 0
        for i, (box, cls_id, conf) in enumerate(zip(boxes, class_ids, confidences)):
            if conf < conf_threshold:
                continue
            if int(cls_id) != target_class:
                continue  # skip other classes

            x1, y1, x2, y2 = map(int, box[:4])
            cropped = image[y1:y2, x1:x2]

            basename = os.path.basename(image_path).split('.')[0]
            out_path = os.path.join(output_dir, f"{basename}_class{target_class}_crop{crop_count+1}.jpg")
            cv2.imwrite(out_path, cropped)
            print(f"✅ Saved: {out_path}")
            crop_count += 1


In [5]:
# crop หลายจานได้

def crop_all_dishes_from_results(results, image_paths, output_dir='cropped', conf_threshold=0.3, class_names=None):
    os.makedirs(output_dir, exist_ok=True)

    for idx, result in enumerate(results):
        image_path = image_paths[idx]
        image = cv2.imread(image_path)

        if image is None:
            print(f"❌ Failed to read {image_path}")
            continue

        try:
            boxes = result.boxes.xyxy.cpu().numpy()   # [x1, y1, x2, y2, conf]
            class_ids = result.boxes.cls.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
        except AttributeError:
            print(f"❌ result[{idx}] missing .boxes attributes")
            continue

        basename = os.path.splitext(os.path.basename(image_path))[0]
        count = 0
        for i, (box, cls_id, conf) in enumerate(zip(boxes, class_ids, confidences)):
            if conf < conf_threshold:
                continue

            x1, y1, x2, y2 = map(int, box[:4])
            cropped = image[y1:y2, x1:x2]

            class_name = f"class{int(cls_id)}"
            if class_names and int(cls_id) < len(class_names):
                class_name = class_names[int(cls_id)]

            save_name = f"{basename}_{class_name}_crop{i+1}.jpg"
            save_path = os.path.join(output_dir, save_name)
            cv2.imwrite(save_path, cropped)
            print(f"✅ Saved: {save_path}")
            count += 1

        if count == 0:
            print(f"⚠️ No dishes detected in {image_path} with confidence > {conf_threshold}")


In [9]:
# แก้pathก้ด้วยนะ ของกุใส่.ละมันหาไม่เจอ
image_dir = "C:/Users/niwat maneewong/OneDrive/Desktop/First/homwork/CS/Thaifood_ObjectDetectionNClassification/datasets/food/traintest/output"
# อันนี้กุลองบางรูปนะ
# image_names = ["plate1.jpg", "plate2.jpg", "plate3.jpg", "plate4.jpg", "plate5.jpg"]
# image_names = ["plate6.jpg","plate8.jpg","plate9.jpg"]
image_names = ["output_plate1.jpg","output_plate2.jpg","output_plate3.jpg"]
image_paths = [os.path.join(image_dir, name) for name in image_names]

# Make sure 'results' is already created with model(image_paths)
# Then call:
results = [model(os.path.join(image_dir, name))[0] for name in image_names]
# crop_from_results(results, image_paths, target_class=1)  # dish2
class_names = ["dish1", "dish2"]
crop_all_dishes_from_results(results, image_paths, class_names=class_names)



image 1/1 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output\output_plate1.jpg: 448x640 1 dish2, 47.4ms
Speed: 7.1ms preprocess, 47.4ms inference, 2.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output\output_plate2.jpg: 448x640 1 dish1, 2 dish2s, 45.2ms
Speed: 5.5ms preprocess, 45.2ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\output\output_plate3.jpg: 448x640 1 dish2, 44.8ms
Speed: 3.9ms preprocess, 44.8ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)
✅ Saved: cropped\output_plate1_dish2_crop1.jpg
✅ Saved: cropped\output_plate2_dish2_crop1.jpg
✅ Saved: cropped\output_plate2_dish1_crop2.jpg
✅ Save

In [8]:
results = model(image_dir)
print(type(results))
print(dir(results))


image 1/9 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\plate\plate1.jpg: 448x640 1 dish2, 45.7ms
image 2/9 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\plate\plate2.jpg: 448x640 1 dish1, 1 dish2, 44.9ms
image 3/9 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\plate\plate3.jpg: 448x640 1 dish1, 1 dish2, 44.9ms
image 4/9 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\plate\plate4.jpg: 448x640 1 dish1, 1 dish2, 44.8ms
image 5/9 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNClassification\datasets\food\traintest\plate\plate5.jpg: 480x640 1 dish2, 48.2ms
image 6/9 C:\Users\niwat maneewong\OneDrive\Desktop\First\homwork\CS\Thaifood_ObjectDetectionNCla

In [ ]:
# ไม่น่าต้องใละ
def yolo_to_xyxy(xc, yc, w, h, img_w, img_h):
    x1 = int((xc - w / 2) * img_w)
    y1 = int((yc - h / 2) * img_h)
    x2 = int((xc + w / 2) * img_w)
    y2 = int((yc + h / 2) * img_h)
    return [x1, y1, x2, y2]